## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Input parameters

In [2]:
trailname = 'gr122' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
# points_per_batch = 25 # Subdivide the trail into batches of this many points
# delta = 0.005 # Tolerance around bounding box per trail section [deg]
delta = 0.008

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Function definitions

In [3]:
def get_dist(X,Y):
    r = [X[0] - Y[0],
         X[1] - Y[1]]
    return np.sqrt(r[0]*r[0] + r[1]*r[1])

In [4]:
def get_p2seg_distance(P,P0,P1):
    v = [P1[0] - P0[0],
         P1[1] - P0[1]]
    w = [P[0] - P0[0],
         P[1] - P0[1]]
    c1 = w[0]*v[0] + w[1]*v[1]
    if c1<=0:
        return get_dist(P,P0)
    c2 = v[0]*v[0] + v[1]*v[1]
    if c2<=c1:
        return get_dist(P,P1)
    b = c1/c2
    Pb = [P0[0] + b*v[0],
          P0[1] + b*v[1]]
    return get_dist(P,Pb)

In [5]:
def get_coords_from_route(route):
    
    # route is a list of [51.2780788,4.051181,51.2781197,4.0511689,4.625675490697481,5.734,'residential',nan,nan]
    xy = []
    for segment in route:
        xy.append([segment[0],segment[1]])
    segment = route[-1]
    xy.append([segment[2],segment[3]]) # adding the last point
    return xy

In [6]:
def get_path_error(route,trail_coords):
    xy = get_coords_from_route(route)
    err = 0
    for point in trail_coords: # for each point from the gpx
        dmin = 999
        for j in range(len(xy)-1): # find the min distance to the pathfind
            dmin = min(dmin,get_p2seg_distance(point,xy[j],xy[j+1]))
        err += dmin*dmin # save the squared error
    return err

## Loading GPX file

In [86]:
trail = gr_utils.get_gpx(trailname)

Loading trail points from <data_input/gr122.gpx>...
Finished loading.


## Gathering ROAD data from OSM network

In [70]:
# Define the range of GPX points to process in the current batch
extra_points_at_start = 15
n_trail = len(trail) # Number of GPX points in the trail
b = 1
# n1 = b*points_per_batch
n1 = max(0,b*points_per_batch - extra_points_at_start) # First point of this batch
n2 = min((b+1)*points_per_batch, n_trail) # Last point of this batch (clipped)
trail_section = trail.loc[n1:n2] # Select that range of GPX points
trail_coords  = gr_mapmatch.trail_to_coords(trail_section) # Convert the points into a list of [lat, lon] pairs

In [71]:
print(f'grabbing n1 = {n1} through n2 = {n2}')

grabbing n1 = 85 through n2 = 200


In [85]:
# network, segment_list = gr_mapmatch.match_batch(trail_section, trail_coords, delta)
lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(trail_section,delta) # Calculate the bounding box
network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM

   Processing street network...


In [73]:
node_list_raw = gr_mapmatch.match_nodes(network, trail_coords) # Match each GPX point to a network node

   Handling GPX point 115 of 115...


## Correcting i0

In [74]:
# Figure out node id of point with i = b*points_per_batch
if n1>0:
    node_id = node_list_raw[extra_points_at_start]
    dvec = []
    for j in range(len(node_list_raw)):
        if node_list_raw[j] == node_id:
            tnode = network['points'].loc[node_list_raw[j]]
            node = [tnode['y'],tnode['x']]
            d = get_dist(trail_coords[j],node)
        else:
            d = 999
        dvec.append(d)
    #     print(f'Point {n1+j} {trail_coords[j]} maps to node {node_list_raw[j]}, d = {d}')
    jjmin = dvec.index(min(dvec))
else:
    jjmin = 0
node_list_use = node_list_raw[jjmin:]
trail_coords_use = trail_coords[jjmin:]

## Road matching algorithm

In [84]:
total_route = []
i0 = 0
i1 = 0

print(f'Performing road matching for GPX points {n1+jjmin} through {n2}...')
while i0<len(node_list_use)-1:
    
    # Start from the first node in the raw node list
    node0 = node_list_use[i0]
    i1 = i0
    
    # Now find the first entry that is different from i0
    while i1<len(node_list_use)-1:
        i1 += 1 # increment counter
        node1 = node_list_use[i1]
        if node1 != node0: # then we found a different entry
            break
            
    # Now find the last occurrence that is the same as i1
    i2 = i1
    if i2<len(node_list_use)-1:
        while i2<len(node_list_use)-1:
            i2 += 1 # increment counter
            node2 = node_list_use[i2]
            if node2 != node1: # then we found a different entry
                i2 -= 1
                break
                
    # --- Correct i2
    # calculate for all GPX points between i0 and i2 the distance to the corresponding node
    d_to_node = []
    tcoords = []
    for j in range(i1,i2+1):
    #     print(node_list_raw[j])
        point = trail_coords_use[j]
        tcoords.append(point)
        temp = network['points'].loc[node_list_use[j]]
        node = [temp['y'],temp['x']]
        d_to_node.append(get_dist(point,node))
    # Find the GPX point closest to node1, that is the last GPX point we consider in this step of the loop
    # So correct i2 and proceerd accordingly
    imin = d_to_node.index(min(d_to_node))
    i2 = i1 + imin
#     print(f'Corrected i2 to {i2}')

    print(f'matching points {i0} through {i2}')

    
    # --- Pathfind the k shortest routes between node0 and node2
    k = 5 # number of routes to find
    route = ox.k_shortest_paths(network['graph'], node0, node1, k)
    paths = []
    for path in route:
        paths.append(path)
        
    # --- Choose the best path using element i0 through i2 from the nodes list
    
    # Grab the corresponding coordinates
    routes = []
    for path in paths:
        segment_list = []
        for j in range(len(path)-1):
            segment_list.extend(gr_mapmatch.get_segments(network, path[j], path[j+1]))
        routes.append(segment_list)

    # Get the error for each route
    err = []
    d = []
    for route in routes:
        err.append(get_path_error(route,trail_coords_use[i0:i2+1]))
        d_osm = 0
        for segment in route:
            d_osm += segment[5] # add d_osm column
        d.append(d_osm)
    dmin = min(d)
    weights = []
    if i2-i0 < 4:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(d[j])
    else:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(err[j]*np.exp(d[j]/dmin))
        
    # Choose the best route
    imin = weights.index(min(weights))
    best_route = routes[imin]
    total_route.extend(best_route)
    
#     print(f'matched points {i0} through {i2}')
#     print(f'matched points {n1+i0+jjmin} through {n1+i2}')
#     gr_mapmatch.print_overwrite(f'\r   ... matched points {n1+i0} through {n1+i1}')
    
    # Afterwards, we set i0 = i2 and repeat
    i0 = i2
    if i2==(len(node_list_use)-1):
        break

Performing road matching for GPX points 100 through 200...
matching points 0 through 1
matching points 1 through 2
matching points 2 through 8
matching points 8 through 9
matching points 9 through 11
matching points 11 through 17
matching points 17 through 20
matching points 20 through 35
matching points 35 through 44
matching points 44 through 61
matching points 61 through 62
matching points 62 through 63
matching points 63 through 80
matching points 80 through 85
matching points 85 through 92
matching points 92 through 100


IndexError: list index out of range

## Plotting the matched trail

In [69]:
# Plot the matched trail
xy = get_coords_from_route(total_route)

# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords_use[mid], zoom_start=14, tiles="OpenStreetMap")

# Draw the GPX route
newline = folium.PolyLine(locations=trail_coords_use, weight=3, color='red')
newline.add_to(chart)

# for point in trail_coords:
#     newmarker = folium.CircleMarker(location=point,radius=2,color='blue')
#     newmarker.add_to(chart)

for point in trail_coords_use:
    newmarker = folium.CircleMarker(location=point,radius=2,color='red')
    newmarker.add_to(chart)

# Draw the matched route
newline = folium.PolyLine(locations=xy, weight=2, color='black')
newline.add_to(chart)
    
# Render the map
filepath = f"cache/temp7.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [32]:
len(trail_coords)

111

In [33]:
len(trail_coords_use)

109

## Quantifying the post-matching error

## Drafts below

In [ ]:
total_route = []
i0 = 0
i1 = 0

print(f'Performing road matching for GPX points {n1} through {n2}...')
while i0<len(node_list_raw)-1:
    
    # Start from the first node in the raw node list
    node0 = node_list_raw[i0]
    i1 = i0
    
    # Now find the first entry that is different from i0
    while i1<len(node_list_raw)-1:
        i1 += 1 # increment counter
        node1 = node_list_raw[i1]
        if node1 != node0: # then we found a different entry
            break
            
    # Now find the last occurrence that is the same as i1
    i2 = i1
    if i2<len(node_list_raw)-1:
        while i2<len(node_list_raw)-1:
            i2 += 1 # increment counter
            node2 = node_list_raw[i2]
            if node2 != node1: # then we found a different entry
                i2 -= 1
                break
                
    # --- Correct i2
    # calculate for all GPX points between i0 and i2 the distance to the corresponding node
    d_to_node = []
    tcoords = []
    for j in range(i1,i2+1):
    #     print(node_list_raw[j])
        point = trail_coords[j]
        tcoords.append(point)
        temp = network['points'].loc[node_list_raw[j]]
        node = [temp['y'],temp['x']]
        d_to_node.append(get_dist(point,node))
    # Find the GPX point closest to node1, that is the last GPX point we consider in this step of the loop
    # So correct i2 and proceerd accordingly
    imin = d_to_node.index(min(d_to_node))
    i2 = i1 + imin
#     print(f'Corrected i2 to {i2}')
    
    # --- Pathfind the k shortest routes between node0 and node2
    k = 5 # number of routes to find
    route = ox.k_shortest_paths(network['graph'], node0, node1, k)
    paths = []
    for path in route:
        paths.append(path)
        
    # --- Choose the best path using element i0 through i2 from the nodes list
    
    # Grab the corresponding coordinates
    routes = []
    for path in paths:
        segment_list = []
        for j in range(len(path)-1):
            segment_list.extend(gr_mapmatch.get_segments(network, path[j], path[j+1]))
        routes.append(segment_list)

    # Get the error for each route
    err = []
    d = []
    for route in routes:
        err.append(get_path_error(route,trail_coords[i0:i2+1]))
        d_osm = 0
        for segment in route:
            d_osm += segment[5] # add d_osm column
        d.append(d_osm)
    dmin = min(d)
    weights = []
    if i2-i0 < 4:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(d[j])
    else:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(err[j]*np.exp(d[j]/dmin))
        
    # Choose the best route
    imin = weights.index(min(weights))
    best_route = routes[imin]
    total_route.extend(best_route)
    
#     print(f'matched points {n1+i0} through {n1+i2}')
    gr_mapmatch.print_overwrite(f'\r   ... matched points {n1+i0} through {n1+i1}')
    
    # Afterwards, we set i0 = i2 and repeat
    i0 = i2
    if i2==(len(node_list_raw)-1):
        break

In [33]:
### Plot the matching from GPX points to nodes
network['points'].loc[node_list_raw[0]]
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=16, tiles="OpenStreetMap")
# Draw the two nodes
n0 = network['points'].loc[node0]
point0 = [n0['y'],n0['x']]
newmarker = folium.CircleMarker(location=point0,radius=3,color='black')
newmarker.add_to(chart)
n1 = network['points'].loc[node1]
point1 = [n1['y'],n1['x']]
newmarker = folium.CircleMarker(location=point1,radius=3,color='black')
newmarker.add_to(chart)
# Draw the k shortest paths
colors = ['red','orange','yellow','green','blue','black']
for j in range(len(routes)):
    xy = gr_plot.get_fullcoords_from_frame(routes[j])
    if j==imin:
        wdt = 3
    else:
        wdt = 1
    newline = folium.PolyLine(locations=xy, weight=wdt, color=colors[j],popup=err[j])
    newline.add_to(chart)
# Draw the GPX points in between
for j in range(i0,i2+1):
    point = trail_coords[j]
    newmarker = folium.CircleMarker(location=point,radius=3,color='red',popup=node_list_raw[j])
    newmarker.add_to(chart)
# Render the map
filepath = f"cache/temp6.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [32]:
route = ox.k_shortest_paths(network['graph'], id0, id1, k)
paths = []
for path in route:
    paths.append(path)

In [68]:
# Grab the corresponding coordinates
routes = []
for path in paths:
    segment_list = []
    for j in range(len(path)-1):
        segment_list.extend(gr_mapmatch.get_segments(network, path[j], path[j+1]))
    fout = f'temp.csv'
    gr_utils.write_batch(fout, segment_list)
    data = pd.read_csv(fout,dtype={'highway':str, 'surface': str, 'tracktype':str})
    routes.append(data)

   Writing outputs to file...
   Writing outputs to file...
   Writing outputs to file...
   Writing outputs to file...
   Writing outputs to file...


In [69]:
### Plot the matching from GPX points to nodes
network['points'].loc[node_list_raw[0]]
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=16, tiles="OpenStreetMap")
# Draw the two nodes
node0 = network['points'].loc[id0]
point0 = [node0['y'],node0['x']]
newmarker = folium.CircleMarker(location=point0,radius=3,color='black')
newmarker.add_to(chart)
node1 = network['points'].loc[id1]
point1 = [node1['y'],node1['x']]
newmarker = folium.CircleMarker(location=point1,radius=3,color='black')
newmarker.add_to(chart)
# Draw the k shortest paths
colors = ['red','orange','yellow','green','blue','black']
for j in range(len(routes)):
    xy = gr_plot.get_fullcoords_from_frame(routes[j])
    newline = folium.PolyLine(locations=xy, weight=3, color=colors[j])
    newline.add_to(chart)
# Render the map
filepath = f"cache/temp5.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

## Plotting

In [8]:
### Plot the matching from GPX points to nodes
network['points'].loc[node_list_raw[0]]
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=16, tiles="OpenStreetMap")
# Draw GPX track
newline = folium.PolyLine(locations=trail_coords, weight=3, color='red')
newline.add_to(chart)
for point in trail_coords:
    newmarker = folium.CircleMarker(location=point,radius=3,color='red')
    newmarker.add_to(chart)
# Draw matched nodes
for node_id in node_list_raw:
    node = network['points'].loc[node_id]
    point = [node['y'],node['x']]
    newmarker = folium.CircleMarker(location=point,radius=3,color='blue')
    newmarker.add_to(chart)
# Render the map
filepath = f"cache/temp4.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [13]:
### Plotting the rendered shortest route
network['points'].loc[node_list_raw[0]]
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=16, tiles="OpenStreetMap")
# Draw GPX track
newline = folium.PolyLine(locations=trail_coords, weight=3, color='red')
newline.add_to(chart)
for point in trail_coords:
    newmarker = folium.CircleMarker(location=point,radius=3,color='red')
    newmarker.add_to(chart)
# Draw matched nodes
for node_id in node_list_raw:
    node = network['points'].loc[node_id]
    point = [node['y'],node['x']]
    newmarker = folium.CircleMarker(location=point,radius=3,color='blue',popup=node_id)
    newmarker.add_to(chart)
# Draw the matching
for j in range(len(trail_coords)):
    point_gpx = trail_coords[j]
    node_id = node_list_raw[j]
    node = network['points'].loc[node_id]
    point = [node['y'],node['x']]
    newline = folium.PolyLine(locations=[point_gpx,point], weight=2, color='black')
    newline.add_to(chart)
# Draw pathfind
# for j in range(len(route_list)-1):
#     node0 = route_list[j]
#     node1 = route_list[j+1]
#     edge = network['edges'].loc[node0,node1]
#     lon = list(edge['geometry'][0].coords.xy[0])
#     lat = list(edge['geometry'][0].coords.xy[1])
#     xy = [[coord[1],coord[0]] for coord in list(zip(lon,lat))]
#     newline = folium.PolyLine(locations=xy, weight=3, color='blue')
#     newline.add_to(chart)
# Render the map
filepath = f"cache/temp5.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [29]:
total

[1.7906131529517318e-06,
 1.1798607727343111e-08,
 1.7869421466577503e-06,
 4.0991360758709733e-07,
 3.7252144758789297e-07]

In [10]:
### Just drawing the two paths
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=14, tiles="OpenStreetMap") 
# Draw GPX track
newline = folium.PolyLine(locations=trail_coords, weight=3, color='red')
newline.add_to(chart)
for point in trail_coords:
    newmarker = folium.CircleMarker(location=point,radius=3,color='red')
    newmarker.add_to(chart)
# Draw map matched frame
xy = gr_plot.get_coords_from_frame(data)
newline = folium.PolyLine(locations=xy, weight=2, color='black')
newline.add_to(chart)
for point in xy:
    newmarker = folium.CircleMarker(location=point,radius=3,color='black')
    newmarker.add_to(chart)
# Render the map
filepath = f"cache/temp.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [13]:
### Calculating the error between the two paths, at all GPX points
# for all gpx points
dmin = []
for point_matched in xy:
    d = 999.0
    for j in range(len(trail_coords)-1):
        d = min(d,get_p2seg_distance(point_matched,trail_coords[j],trail_coords[j+1]))
    dmin.append(1e6*d)      

In [14]:
### Just drawing the two paths
mytol = 100
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=14, tiles="OpenStreetMap") 
# Draw GPX points in red
newline = folium.PolyLine(locations=trail_coords, weight=3, color='red')
newline.add_to(chart)
for point in trail_coords:
    newmarker = folium.CircleMarker(location=point,radius=3,color='red')
    newmarker.add_to(chart)
# Draw map matched points
xy = gr_plot.get_coords_from_frame(data)
newline = folium.PolyLine(locations=xy, weight=2, color='black')
newline.add_to(chart)
for j in range(len(xy)):
    point = xy[j]
    d = dmin[j]
    color = 'black'
    if d<mytol: # good points are black
        color = 'black'
    else: # bad points are blue
        color = 'blue'
    newmarker = folium.CircleMarker(location=point,radius=3,color=color,popup=np.round(d))
    newmarker.add_to(chart)
# Render the map
filepath = f"cache/temp.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)

In [15]:
### Just drawing the two paths
mytol = 100
# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=14, tiles="OpenStreetMap") 
# Draw GPX points in red
# newline = folium.PolyLine(locations=trail_coords, weight=1, color='red')
# newline.add_to(chart)
# Render the map
filepath = f"cache/temp2.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)